In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
chunks = pd.read_csv('/content/drive/MyDrive/data_new_project/Copy of crediarc_transactions_full.csv',
                       chunksize=500000)

In [3]:
trans_df = pd.concat(chunks, ignore_index=True)

<ipython-input-3-8de9670f1503>:1: DtypeWarning: Columns (6,8,10,11,12,16,17,18) have mixed types. Specify dtype option on import or set low_memory=False.
  trans_df = pd.concat(chunks, ignore_index=True)
<ipython-input-3-8de9670f1503>:1: DtypeWarning: Columns (6,8,10,11,12,16,17,18) have mixed types. Specify dtype option on import or set low_memory=False.
  trans_df = pd.concat(chunks, ignore_index=True)
<ipython-input-3-8de9670f1503>:1: DtypeWarning: Columns (6,8,10,11,12,16,17,18) have mixed types. Specify dtype option on import or set low_memory=False.
  trans_df = pd.concat(chunks, ignore_index=True)
<ipython-input-3-8de9670f1503>:1: DtypeWarning: Columns (6,8,10,11,12,16,17,18) have mixed types. Specify dtype option on import or set low_memory=False.
  trans_df = pd.concat(chunks, ignore_index=True)
<ipython-input-3-8de9670f1503>:1: DtypeWarning: Columns (6,8,10,11,12,16,17,18) have mixed types. Specify dtype option on import or set low_memory=False.
  trans_df = pd.concat(chunks,

In [4]:
accounts_df = pd.read_csv('/content/drive/MyDrive/data_new_project/crediarc_accounts_full_nn.csv')

In [5]:
trans_df['TRANSACTION_ON'].dropna(inplace=True)

In [6]:
trans_df['TRANSACTION_ON'] = pd.to_datetime(trans_df['TRANSACTION_ON'])

In [7]:
trans_df['TRANSACTION_ON'].min()

Timestamp('2023-02-08 08:09:13')

In [8]:
trans_df['TRANSACTION_ON'].max()

Timestamp('2025-05-14 10:47:44')

In [9]:
trans_df['TRANSACTION_ON'].dropna(inplace=True)

In [10]:
cutoff_date = pd.to_datetime("2024-01-01")

In [11]:
past_txns = trans_df[trans_df["TRANSACTION_ON"] < cutoff_date]

In [12]:
future_txns = trans_df[trans_df["TRANSACTION_ON"] >= cutoff_date]

In [13]:
future_refunds = (
    future_txns.groupby("COMPANY_ID")["AMOUNT_REFUNDED"]
    .sum()
    .reset_index()
)

In [14]:
future_refunds

,COMPANY_ID,AMOUNT_REFUNDED
0,527ebb1db090f2cc4c00035d,0.0
1,52a7342b697df18c89000ec7,0.0
2,52df2956840d2e897c000fbf,0.0
3,532decf83237c56b3f000329,0.0
4,5335e2fd137e79d87e000142,0.0
...,...,...
97691,6822a63fe0469f00196a5829,0.0
97692,68238396a649280019d1f106,0.0
97693,682385b45b14c7001fb1b74a,100.0
97694,68238934f1dc5f0026e11273,0.0


In [15]:
future_refunds["refund_flag"] = (future_refunds["AMOUNT_REFUNDED"] > 0).astype(int)
future_refunds = future_refunds[["COMPANY_ID", "refund_flag"]]

In [16]:
features = past_txns.groupby("COMPANY_ID").agg(
    num_transactions=("FF_ID", "count"),
    total_payment_usd=("PAYMENT_AMOUNT", "sum"),  # Assume all in USD now
    total_refunded_usd=("AMOUNT_REFUNDED", "sum"),
    avg_payment=("PAYMENT_AMOUNT", "mean"),
    max_payment=("PAYMENT_AMOUNT", "max"),
    num_refunds=("AMOUNT_REFUNDED", lambda x: (x > 0).sum()),
    num_disputes=("IS_DISPUTE", "sum"),
).reset_index()

In [17]:
features

,COMPANY_ID,num_transactions,total_payment_usd,total_refunded_usd,avg_payment,max_payment,num_refunds,num_disputes
0,527ebb1db090f2cc4c00035d,13,46250.00,0.0,3557.692308,9500.00,0,0
1,52a7342b697df18c89000ec7,15,26090.00,0.0,1739.333333,3500.00,0,0
2,52df2956840d2e897c000fbf,6,8380.31,0.0,1396.718333,1700.58,0,0
3,532decf83237c56b3f000329,25,62179.75,0.0,2487.190000,7250.00,0,0
4,5335e2fd137e79d87e000142,16,41229.83,1399483.0,2576.864375,7245.00,5,0
...,...,...,...,...,...,...,...,...
85465,65905a157a0aab48a3073eaf,1,90.00,0.0,90.000000,90.00,0,0
85466,6590686631b6d800322545d9,1,200.00,0.0,200.000000,200.00,0,0
85467,65909a9a38a167029eabe5de,1,25.00,0.0,25.000000,25.00,0,0
85468,659113a6db45ca06a22026fd,1,5.00,0.0,5.000000,5.00,0,0


In [18]:
features["refund_rate"] = features["total_refunded_usd"] / features["total_payment_usd"].replace(0, 1)

In [19]:
past_txns["TRANSACTION_ON"] = pd.to_datetime(past_txns["TRANSACTION_ON"])

# Reference date (cutoff)
cutoff = pd.to_datetime("2024-12-01")

recent_txns = past_txns.groupby("COMPANY_ID").agg(
    last_txn_date=("TRANSACTION_ON", "max"),
    first_txn_date=("TRANSACTION_ON", "min")
).reset_index()

recent_txns["days_since_last_txn"] = (cutoff - recent_txns["last_txn_date"]).dt.days
recent_txns["active_days"] = (recent_txns["last_txn_date"] - recent_txns["first_txn_date"]).dt.days

<ipython-input-19-f3e0071a200b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  past_txns["TRANSACTION_ON"] = pd.to_datetime(past_txns["TRANSACTION_ON"])


In [20]:
recent_txns = recent_txns[['COMPANY_ID','days_since_last_txn','active_days']]
features = features.merge(recent_txns,on='COMPANY_ID',how='left')

In [21]:
txn_counts = past_txns.groupby("COMPANY_ID").agg(
    total_txns=("FF_ID", "count"),
    txn_period_days=("TRANSACTION_ON", lambda x: (x.max() - x.min()).days + 1)
).reset_index()

txn_counts["txns_per_month"] = txn_counts["total_txns"] / (txn_counts["txn_period_days"] / 30.0)

In [22]:
features = features.merge(txn_counts,on='COMPANY_ID',how='left')

In [23]:
last_90d_cutoff = cutoff - pd.Timedelta(days=90)
recent = past_txns[past_txns["TRANSACTION_ON"] >= last_90d_cutoff]

recent_features = recent.groupby("COMPANY_ID").agg(
    recent_txns=("FF_ID", "count"),
    recent_total_paid=("PAYMENT_AMOUNT", "sum"),
    recent_refunds=("AMOUNT_REFUNDED", "sum")
).reset_index()

recent_features["recent_refund_ratio"] = recent_features["recent_refunds"] / recent_features["recent_total_paid"].replace(0, 1)

In [24]:
features = features.merge(recent_features,on='COMPANY_ID',how='left')

In [25]:
vendor_info = accounts_df[[
    "COMPANY_ID", "COMPANY_TYPE", "MEMBER_SEGMENT_ATV", "MONTHS_AGE",
    "NO_OF_EMPLOYEES", "GPV_2YEARS", "PROJECTS_2YEARS", "PAYMENTS_NOT_MANUAL"
]]

In [26]:
def generate_account_features(df):

    numeric_cols = ["MONTHS_AGE", "NO_OF_EMPLOYEES", "PAYMENTS_NOT_MANUAL", "GPV_2YEARS", "PROJECTS_2YEARS"]

    # One-hot encode categorical columns
    df = pd.get_dummies(df, columns=["COMPANY_TYPE", "MEMBER_SEGMENT_ATV"], dummy_na=True)

# Convert all dummy columns to int
    dummy_cols = df.columns.difference(["COMPANY_ID", "MONTHS_AGE", "NO_OF_EMPLOYEES",
                                    "PAYMENTS_NOT_MANUAL", "GPV_2YEARS", "PROJECTS_2YEARS"])
    df[dummy_cols] = df[dummy_cols].astype(int)

    return df

In [27]:
account_features = generate_account_features(vendor_info)

In [28]:
account_features

,COMPANY_ID,MONTHS_AGE,NO_OF_EMPLOYEES,GPV_2YEARS,PROJECTS_2YEARS,PAYMENTS_NOT_MANUAL,COMPANY_TYPE_Agency / Franchise,COMPANY_TYPE_Blogger,COMPANY_TYPE_Bookkeeping / Accounting,COMPANY_TYPE_Business Consultant,...,COMPANY_TYPE_Virtual Assistant,COMPANY_TYPE_Visual Artist,COMPANY_TYPE_Web Designer,COMPANY_TYPE_nan,MEMBER_SEGMENT_ATV_200K+,MEMBER_SEGMENT_ATV_20K - 50K,MEMBER_SEGMENT_ATV_50K - 200K,MEMBER_SEGMENT_ATV_Less than 20K,MEMBER_SEGMENT_ATV_Unknown,MEMBER_SEGMENT_ATV_nan
0,527ebb1db090f2cc4c00035d,79,NaN,152625.00,15,34,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,52a7342b697df18c89000ec7,79,2,53116.57,14,34,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,52df2956840d2e897c000fbf,79,NaN,13560.31,6,9,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,532decf83237c56b3f000329,79,NaN,134359.50,19,26,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,5335e2fd137e79d87e000142,79,NaN,64359.43,23,31,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115831,681d51eb77273500196b086c,0,NaN,1372.99,1,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
115832,681e366d2c69f300191f2a40,0,NaN,1200.00,1,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
115833,681eb82feda79300251e112b,0,NaN,299.70,1,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
115834,681ef9458f13b9002da20040,0,NaN,150.00,2,2,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [29]:
features = features.merge(account_features,on='COMPANY_ID',how='left')

In [30]:
features

,COMPANY_ID,num_transactions,total_payment_usd,total_refunded_usd,avg_payment,max_payment,num_refunds,num_disputes,refund_rate,days_since_last_txn,...,COMPANY_TYPE_Virtual Assistant,COMPANY_TYPE_Visual Artist,COMPANY_TYPE_Web Designer,COMPANY_TYPE_nan,MEMBER_SEGMENT_ATV_200K+,MEMBER_SEGMENT_ATV_20K - 50K,MEMBER_SEGMENT_ATV_50K - 200K,MEMBER_SEGMENT_ATV_Less than 20K,MEMBER_SEGMENT_ATV_Unknown,MEMBER_SEGMENT_ATV_nan
0,527ebb1db090f2cc4c00035d,13,46250.00,0.0,3557.692308,9500.00,0,0,0.000000,354,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,52a7342b697df18c89000ec7,15,26090.00,0.0,1739.333333,3500.00,0,0,0.000000,353,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,52df2956840d2e897c000fbf,6,8380.31,0.0,1396.718333,1700.58,0,0,0.000000,443,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,532decf83237c56b3f000329,25,62179.75,0.0,2487.190000,7250.00,0,0,0.000000,395,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,5335e2fd137e79d87e000142,16,41229.83,1399483.0,2576.864375,7245.00,5,0,33.943458,356,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85465,65905a157a0aab48a3073eaf,1,90.00,0.0,90.000000,90.00,0,0,0.000000,335,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
85466,6590686631b6d800322545d9,1,200.00,0.0,200.000000,200.00,0,0,0.000000,335,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
85467,65909a9a38a167029eabe5de,1,25.00,0.0,25.000000,25.00,0,0,0.000000,336,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
85468,659113a6db45ca06a22026fd,1,5.00,0.0,5.000000,5.00,0,0,0.000000,335,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
df = features.merge(future_refunds,on='COMPANY_ID',how='left')

In [32]:
df = df.dropna(subset='refund_flag')

In [33]:
df.drop(['NO_OF_EMPLOYEES'],axis=1,inplace=True)

In [34]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

# 1. Prepare features and target
X = df.drop(columns=["refund_flag"])
y = df["refund_flag"]

# 2. Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

train_companies = X_train.pop('COMPANY_ID')
test_companies = X_test.pop('COMPANY_ID')

n_pos = np.sum(y_train == 1)
n_neg = np.sum(y_train == 0)

scale_pos_weight = n_neg / n_pos

# 3. Fit XGBoost
xgb_model = XGBClassifier(
    n_estimators=100,
    max_depth=5,
    learning_rate=0.1,
    subsample=1,
    eval_metric="aucpr",
    scale_pos_weight=scale_pos_weight*0.8,
    random_state=42
)
xgb_model.fit(X_train, y_train)

# 4. Evaluate
y_pred = xgb_model.predict(X_test)
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Confusion Matrix:
 [[7667 1761]
 [1185 2209]]

Classification Report:
               precision    recall  f1-score   support

         0.0       0.87      0.81      0.84      9428
         1.0       0.56      0.65      0.60      3394

    accuracy                           0.77     12822
   macro avg       0.71      0.73      0.72     12822
weighted avg       0.78      0.77      0.78     12822



In [35]:
from sklearn.metrics import roc_auc_score

# Predict probabilities for the positive class (refund)
y_probs = xgb_model.predict_proba(X_test)[:, 1]

# Calculate ROC AUC score
roc_auc = roc_auc_score(y_test, y_probs)
print(f"ROC AUC Score: {roc_auc:.4f}")

ROC AUC Score: 0.8168


In [36]:
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score

thresholds = np.arange(0.1, 1.0, 0.1)
y_probs = xgb_model.predict_proba(X_test)[:, 1]

print(f"{'Threshold':<10}{'Precision':<10}{'Recall':<10}{'F1-score':<10}")
for thresh in thresholds:
    y_pred_thresh = (y_probs >= thresh).astype(int)
    precision = precision_score(y_test, y_pred_thresh)
    recall = recall_score(y_test, y_pred_thresh)
    f1 = f1_score(y_test, y_pred_thresh)
    print(f"{thresh:<10.1f}{precision:<10.3f}{recall:<10.3f}{f1:<10.3f}")

Threshold Precision Recall    F1-score  
0.1       0.292     0.984     0.450     
0.2       0.355     0.932     0.514     
0.3       0.426     0.855     0.568     
0.4       0.487     0.763     0.595     
0.5       0.556     0.651     0.600     
0.6       0.632     0.527     0.575     
0.7       0.734     0.388     0.508     
0.8       0.836     0.253     0.388     
0.9       0.937     0.159     0.271     


In [37]:
df["predicted_refund_risk"] = xgb_model.predict_proba(X.drop('COMPANY_ID',axis=1))[:,1]

In [38]:
df

,COMPANY_ID,num_transactions,total_payment_usd,total_refunded_usd,avg_payment,max_payment,num_refunds,num_disputes,refund_rate,days_since_last_txn,...,COMPANY_TYPE_Web Designer,COMPANY_TYPE_nan,MEMBER_SEGMENT_ATV_200K+,MEMBER_SEGMENT_ATV_20K - 50K,MEMBER_SEGMENT_ATV_50K - 200K,MEMBER_SEGMENT_ATV_Less than 20K,MEMBER_SEGMENT_ATV_Unknown,MEMBER_SEGMENT_ATV_nan,refund_flag,predicted_refund_risk
0,527ebb1db090f2cc4c00035d,13,46250.00,0.0,3557.692308,9500.00,0,0,0.000000,354,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.209866
1,52a7342b697df18c89000ec7,15,26090.00,0.0,1739.333333,3500.00,0,0,0.000000,353,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.204849
2,52df2956840d2e897c000fbf,6,8380.31,0.0,1396.718333,1700.58,0,0,0.000000,443,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.106869
3,532decf83237c56b3f000329,25,62179.75,0.0,2487.190000,7250.00,0,0,0.000000,395,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.122446
4,5335e2fd137e79d87e000142,16,41229.83,1399483.0,2576.864375,7245.00,5,0,33.943458,356,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.763120
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85463,6590487bec62ca47f73dd0b6,1,1.00,0.0,1.000000,1.00,0,0,0.000000,335,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.030810
85465,65905a157a0aab48a3073eaf,1,90.00,0.0,90.000000,90.00,0,0,0.000000,335,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.083517
85466,6590686631b6d800322545d9,1,200.00,0.0,200.000000,200.00,0,0,0.000000,335,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.233195
85467,65909a9a38a167029eabe5de,1,25.00,0.0,25.000000,25.00,0,0,0.000000,336,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.045217


In [39]:
def recommend_collateral_dynamic_weight_extended(row,
                                                weight_pred_min=0.3,
                                                weight_pred_max=0.9,
                                                min_prob_threshold=0.4):
    pred_prob = row["predicted_refund_risk"]
    hist_rate = row["refund_rate"]

    # Handle vendors with few transactions (fallback)
    if row["num_transactions"] < 5:
        # Find MEMBER_SEGMENT_ATV columns
        atv_cols = [c for c in df.columns if c.startswith("MEMBER_SEGMENT_ATV_")]
        atv_col = next((c for c in atv_cols if row[c] == 1), None)

        # Find COMPANY_TYPE columns
        comp_cols = [c for c in df.columns if c.startswith("COMPANY_TYPE_")]
        comp_col = next((c for c in comp_cols if row[c] == 1), None)

        # Try MEMBER_SEGMENT_ATV first
        if comp_col:
            hist_rate = df[df[comp_col] == 1]["refund_rate"].mean()
        # Then try COMPANY_TYPE
        elif atv_col:
            hist_rate = df[df[atv_col] == 1]["refund_rate"].mean()
        # Else fallback to global mean
        else:
            hist_rate = df["refund_rate"].mean()

    if pred_prob < min_prob_threshold:
        return (round(min_prob_threshold*0.2 / 0.05) * 0.05)

    scale = (pred_prob - min_prob_threshold) / (1 - min_prob_threshold)
    weight_pred = weight_pred_min + scale * (weight_pred_max - weight_pred_min)
    weight_hist = 1 - weight_pred

    est_risk = weight_pred * pred_prob + weight_hist * hist_rate

    collateral_pct = np.clip(est_risk, 0.05, 0.50)
    collateral_pct_rounded = round(collateral_pct / 0.05) * 0.05

    return collateral_pct_rounded

df["recommended_collateral_pct"] = df.apply(recommend_collateral_dynamic_weight_extended, axis=1)

In [42]:
df[['COMPANY_ID','predicted_refund_risk','recommended_collateral_pct']].to_csv('risk_probability_and_recommended_collateral_perc.csv')